# Generating Gamma Surfaces & Stacking Faults

Gamma surfaces (also called generalised stacking faults) and stacking faults are forms of crystal defects formed by sliding two slabs of a crystal against each other along a given plane. Often, these defects have metastable states (also called stacking faults), which can exist at grain boundaries or between two partial dislocations. Here, we deal with the full pathway from a complete crystal, sliding through any metastable points until we return back to the perfect crystal again. 

For a stacking fault, the slabs are moved along a single line, whereas for gamma surfaces this is the full plane (hence generalised stacking fault). `matscipy.gamma_surface` provides two classes - `GammaSurface` and `StackingFault` - in order to model these systems, including utility methods for relaxing and plotting the associated potential energy surfaces.


There are two main methodologies in generating structures representing gamma surfaces. Both methods start with first generating a structure that has a basis vector parallel to the miller index of the required gamma surface, which we achieve using `ase.build.cut` and `ase.build.rotate`.

The first and most intuitive method involves splitting a supercell of this rotated structure into an upper and lower slab, and translating the top slab relative to the bottom. In the fully periodic case, this creates two stacking faults in the cell, at both the upper and lower boundaries of the top slab. We could instead choose to leave free surfaces above the top slab and below the bottom slab by removing the periodicity in the direction of the surface normal, which leaves us with a single gamma surface between the slabs.

The second method method is more efficient as it reduces the number of atoms required, but is less intuitive to understand. Instead of moving atoms, we instead modify the cell vectors without moving the atoms. In doing this, we translate the periodic image of the bottom of the structure relative to the top of the structure, and vice versa, thus skipping the need to have two slabs. This method always produces a structure containing a single gamma surface, which is always at the cell boundaries. In order to model the free-surface approach, we can add vacuum to the center of the cell, which creates a similar two slab setup as method 1, but this time in a periodic cell.